# Backpropagation

### Import libraries

In [240]:
import math
import random
import numpy as np

### Utility functions

Calculate a random number rand which $a \leq rand < b$

In [241]:
random.seed(0)
def rand(a, b):
    return (b-a)*random.random() + a

Make a matrix

In [242]:
def make_matrix(I, J, fill=0.0):
    return np.full((I, J), fill)

Sigmoid

In [243]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
    # return math.tanh(x)

Sigmoid derivative

In [244]:
def derivative_sigmoid(y):
    return y * (1 - y)
    # return 1.0 - y**2

#### Network class

In [245]:
class Network:
    def __init__(self, input_size, hidden_size, output_size):
        self.input_size = input_size + 1 # +1 for bias node
        self.hidden_size = hidden_size
        self.output_size = output_size

        # activations for nodes
        self.input_activation = [1.0] * self.input_size
        self.hidden_activation = [1.0] * self.hidden_size
        self.output_activation = [1.0] * self.output_size

        # create matrix of weights
        self.input_weight = make_matrix(self.input_size, self.hidden_size)
        self.output_weight = make_matrix(self.hidden_size, self.output_size)
        # instantiates the matrix with random values
        for i in range(self.input_size):
            for j in range(self.hidden_size):
                self.input_weight[i][j] = rand(-0.2, 0.2)
        for j in range(self.hidden_size):
            for k in range(self.output_size):
                self.output_weight[j][k] = rand(-2.0, 2.0)

        # last change in weights for momentum
        self.input_change = make_matrix(self.input_size, self.hidden_size)
        self.output_change = make_matrix(self.hidden_size, self.output_size)

    def update(self, inputs):
        if len(inputs) != self.input_size - 1:
            raise ValueError('Inputs out of bounds')

        # input activations
        for i in range(self.input_size - 1):
            self.input_activation[i] = inputs[i]

        # hidden activations
        for j in range(self.hidden_size):
            sum = 0.0
            for i in range(self.input_size):
                sum = sum + self.input_activation[i] * self.input_weight[i][j]
            self.hidden_activation[j] = sigmoid(sum)

        # output activations
        for k in range(self.output_size):
            sum = 0.0
            for j in range(self.hidden_size):
                sum = sum + self.hidden_activation[j] * self.output_weight[j][k]
            self.output_activation[k] = sigmoid(sum)

        return self.output_activation[:]


    def backpropagation(self, targets, N, M):
        if len(targets) != self.output_size:
            raise ValueError('Targets out of bounds')

        # calculate error terms for output
        output_deltas = [0.0] * self.output_size
        for k in range(self.output_size):
            error = targets[k] - self.output_activation[k]
            output_deltas[k] = derivative_sigmoid(self.output_activation[k]) * error

        # calculate error terms for hidden
        hidden_deltas = [0.0] * self.hidden_size
        for j in range(self.hidden_size):
            error = 0.0
            for k in range(self.output_size):
                error = error + output_deltas[k] * self.output_weight[j][k]
            hidden_deltas[j] = derivative_sigmoid(self.hidden_activation[j]) * error

        # update output weights
        for j in range(self.hidden_size):
            for k in range(self.output_size):
                change = output_deltas[k] * self.hidden_activation[j]
                self.output_weight[j][k] = self.output_weight[j][k] + N * change + M * self.output_change[j][k]
                self.output_change[j][k] = change

        # update input weights
        for i in range(self.input_size):
            for j in range(self.hidden_size):
                change = hidden_deltas[j]*self.input_activation[i]
                self.input_weight[i][j] = self.input_weight[i][j] + N * change + M * self.input_change[i][j]
                self.input_change[i][j] = change

        # calculate error
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5*(targets[k] - self.output_activation[k])**2
        return error

    def test(self, patterns):
        for p in patterns:
            print(p[0], '->', self.update(p[0]))

    def weights(self):
        print('Input weights:')
        for i in range(self.input_size):
            print(self.input_weight[i])
        print()
        print('Output weights:')
        for j in range(self.hidden_size):
            print(self.output_weight[j])

    def train(self, patterns, iterations=10000, N=0.5, M=0.1):
        # N: learning rate
        # M: momentum factor
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backpropagation(targets, N, M)
            if i % 100 == 0:
                print('error %-.5f' % error)

#### Create dataset for testing

Dataset for AND function

In [246]:
ds_and = [
    [[0,0], [0]],
    [[0,1], [0]],
    [[1,0], [0]],
    [[1,1], [1]]
]

Dataset for OR function

In [247]:
ds_or = [
    [[0,0], [0]],
    [[0,1], [1]],
    [[1,0], [1]],
    [[1,1], [1]]
]

Dataset for XOR function

In [248]:
ds_xor = [
    [[0,0], [0]],
    [[0,1], [1]],
    [[1,0], [1]],
    [[1,1], [0]]
]

### Run train and test

In [249]:
n = int(input("Insert number of inputs (>= 2)"))
inputs = np.array([])
for _ in range(n):
  value = int(input("Insert 0 or 1"))
  inputs = np.append(inputs, value)
  
gate = input("Type \"AND\", \"OR\" or \"XOR\"")

if gate == "AND":
  ds = ds_and
elif gate == "OR":
  ds = ds_or
elif gate == "XOR":
  ds = ds_xor
else:
  print("Invalid option")

In [250]:
# create a network with two input, two hidden, and one output nodes
nw = Network(2, 2, 1)

nw.train(ds)
nw.test(ds)

error 0.55099
error 0.19106
error 0.05764
error 0.02400
error 0.01356
error 0.00903
error 0.00661
error 0.00514
error 0.00416
error 0.00348
error 0.00297
error 0.00259
error 0.00229
error 0.00204
error 0.00184
error 0.00168
error 0.00153
error 0.00141
error 0.00131
error 0.00122
error 0.00114
error 0.00107
error 0.00101
error 0.00095
error 0.00090
error 0.00085
error 0.00081
error 0.00077
error 0.00074
error 0.00071
error 0.00068
error 0.00065
error 0.00063
error 0.00060
error 0.00058
error 0.00056
error 0.00054
error 0.00052
error 0.00051
error 0.00049
error 0.00048
error 0.00046
error 0.00045
error 0.00044
error 0.00042
error 0.00041
error 0.00040
error 0.00039
error 0.00038
error 0.00037
error 0.00036
error 0.00035
error 0.00035
error 0.00034
error 0.00033
error 0.00032
error 0.00032
error 0.00031
error 0.00030
error 0.00030
error 0.00029
error 0.00029
error 0.00028
error 0.00028
error 0.00027
error 0.00027
error 0.00026
error 0.00026
error 0.00025
error 0.00025
error 0.00024
error 